# 全量用户的手机号码

In [1]:
"""
Connect senior:Mainland
"""
import pandas as pd
import json
import requests
def ssql(sql):
    admin_token = '2cebeb5c7edac5ef0d7c5a640e69fc7d43bf2cd6a24ce43dfee3dd33662c0bab'
    url = 'http://sensor.wb-intra.com/api/sql/query?token=%s&project=production' % admin_token
    data = {'q': sql, 'format': 'json'}
    req = requests.post(url,data)
    req_dec = req.content.decode()
    try:
        req_json = json.loads('[' + req_dec.replace('\n', ',')[:-1] + ']')
        df_d_id = pd.DataFrame(req_json)
        return df_d_id
    except:
        print(req_dec)

In [2]:
import pandas as pd
import numpy as np
"""
Connect Aliyun and commit MySQL
"""
 
import requests
import json
 
 
def aliq(db, req):
    url = 'http://localhost:8008/aliq?sql=%s&db=%s' % (requests.utils.quote(req), db)
    resp = requests.get(url)
    res = resp.content.decode()
    js = json.loads(res)
    return js

In [3]:
df_wolf = ssql("""
    -- 狼人杀流失各个游戏等级内的百分之5的用户哈

SELECT CASE
           WHEN t1.gap >= 0
                AND t1.gap <5 THEN '3-5'
           WHEN t1.gap >=5
                AND t1.gap < 8 THEN '5-8'
           WHEN t1.gap >=8
                AND t1.gap < 15 THEN '8-15'
           WHEN t1.gap >=15
                AND t1.gap < 30 THEN '15-30'
           WHEN t1.gap >=30
                AND t1.gap < 90 THEN '30-90'
           ELSE '90-180天'
       END AS "流失天数",
       t1.game_level,
       t1.distinct_id
FROM
  (SELECT t0.*,
          datediff(now(),t0.date)AS gap
   FROM
     (SELECT date,gameTypeId,
                  distinct_id,
                  game_level,
                  time,
                  row_number()over(partition BY gameTypeId, distinct_id
                                   ORDER BY time DESC) AS rank
      FROM events
      WHERE event = 'gameOver'
        AND gameTypeId = 1004
        AND date BETWEEN current_date() - interval '180' DAY AND current_date() - interval '1' DAY)t0
   JOIN
     (SELECT second_id
      FROM users
      WHERE is_real_name_phone = 'yes'
      GROUP BY 1)t2 ON t0.distinct_id = t2.second_id
   WHERE t0.rank = 1)t1
GROUP BY 1,
         2,
         3
""")
wolf_user_list = list(df_wolf.distinct_id)
group_user_dict = {str(i) : [] for i in range(100)}
for i in wolf_user_list:
    no = str(int(i[-2:]))
    group_user_dict[no].append(i)

In [4]:
all_info = []
for i in range(100):    
    if group_user_dict[str(i)]:
        table = 'user_phone_%s' % str(i)

        user_list = group_user_dict[str(i)]
        if len(user_list) == 1:
            user_list.append('123')
        sql = """
            select uid, phone_number
            from {table}
            where uid in{user_list}
        """.format(table=table, user_list=tuple(user_list))
        
        data = aliq('spy', sql)
        if data[0]:
            all_info.extend(data)
df_phone = pd.DataFrame.from_dict([{'distinct_id': str(i[0]), 'phone_number': str(i[1])} for i in all_info])
df_merge = pd.merge(df_wolf, df_phone, on='distinct_id', how='left')
df_merge

,流失天数,distinct_id,game_level,phone_number
0,90-180天,21629408,6.0,8613894612455
1,90-180天,132166921,1.0,8615144275076
2,90-180天,75328404,6.0,8617690928262
3,15-30,136978135,3.0,8618544359558
4,30-90,121481846,1.0,8618524624753
...,...,...,...,...
1127874,15-30,47367838,2.0,8613072232323
1127875,30-90,12652670,3.0,8618352836867
1127876,90-180天,58134343,2.0,8618240330906
1127877,30-90,58420801,15.0,8615175900533


In [44]:
df_copy = df_merge.copy()

In [45]:
df_copy['phone_number'] = df_copy['phone_number'].apply(lambda x:str(x)[2:])
df_copy

,流失天数,distinct_id,game_level,phone_number
0,90-180天,21629408,6.0,13894612455
1,90-180天,132166921,1.0,15144275076
2,90-180天,75328404,6.0,17690928262
3,15-30,136978135,3.0,18544359558
4,30-90,121481846,1.0,18524624753
...,...,...,...,...
1127874,15-30,47367838,2.0,13072232323
1127875,30-90,12652670,3.0,18352836867
1127876,90-180天,58134343,2.0,18240330906
1127877,30-90,58420801,15.0,15175900533


In [46]:
import re

In [47]:
def re_op(x):
    a = re.findall('^134[0-8]\d{7}$|^(?:13[5-9]|147|15[0-27-9]|178|1703|1705|1706|18[2-478])\d{7,8}$',str(x))
    if a:
        return a[0]
    else:
        return ''

In [48]:
# df_copy['phone_number'] = df_copy['phone_number'].apply(lambda x: if re.findall('^134[0-8]\d{7}$|^(?:13[5-9]|147|15[0-27-9]|178|1703|1705|1706|18[2-478])\d{7,8}$',str(x)) else '')
df_copy['phone_number'] = df_copy['phone_number'].map(re_op)
df_copy

,流失天数,distinct_id,game_level,phone_number
0,90-180天,21629408,6.0,13894612455
1,90-180天,132166921,1.0,15144275076
2,90-180天,75328404,6.0,
3,15-30,136978135,3.0,
4,30-90,121481846,1.0,
...,...,...,...,...
1127874,15-30,47367838,2.0,
1127875,30-90,12652670,3.0,18352836867
1127876,90-180天,58134343,2.0,18240330906
1127877,30-90,58420801,15.0,15175900533


In [49]:
df_copy = df_copy[df_copy.phone_number!='']
df_copy

,流失天数,distinct_id,game_level,phone_number
0,90-180天,21629408,6.0,13894612455
1,90-180天,132166921,1.0,15144275076
6,90-180天,15560703,19.0,13551039011
7,30-90,122850156,6.0,18750186809
13,15-30,42072515,1.0,15013016149
...,...,...,...,...
1127873,30-90,71681466,16.0,18857448863
1127875,30-90,12652670,3.0,18352836867
1127876,90-180天,58134343,2.0,18240330906
1127877,30-90,58420801,15.0,15175900533


In [22]:
df_copy['流失天数'].unique()

array(['90-180天', '30-90', '15-30', '8-15', '3-5', '5-8'], dtype=object)

In [50]:
df_copy.reset_index(drop=True)

,流失天数,distinct_id,game_level,phone_number
0,90-180天,21629408,6.0,13894612455
1,90-180天,132166921,1.0,15144275076
2,90-180天,15560703,19.0,13551039011
3,30-90,122850156,6.0,18750186809
4,15-30,42072515,1.0,15013016149
...,...,...,...,...
606923,30-90,71681466,16.0,18857448863
606924,30-90,12652670,3.0,18352836867
606925,90-180天,58134343,2.0,18240330906
606926,30-90,58420801,15.0,15175900533


In [51]:
df_sample = df_copy.sample(frac=0.05,replace=False)
df_sample

,流失天数,distinct_id,game_level,phone_number
228861,90-180天,149598890,7.0,13913899872
556389,90-180天,103303226,3.0,13601020320
158245,30-90,123012545,3.0,15852469399
935799,90-180天,130874531,5.0,15765002702
1008329,30-90,130406973,3.0,15283000648
...,...,...,...,...
148757,90-180天,59474476,3.0,15906075118
192283,90-180天,117412985,11.0,13472290404
47971,30-90,86498277,20.0,18742522905
654783,30-90,30642960,6.0,18842211875


In [52]:
df_sample.reset_index(drop = True)

,流失天数,distinct_id,game_level,phone_number
0,90-180天,149598890,7.0,13913899872
1,90-180天,103303226,3.0,13601020320
2,30-90,123012545,3.0,15852469399
3,90-180天,130874531,5.0,15765002702
4,30-90,130406973,3.0,15283000648
...,...,...,...,...
30341,90-180天,59474476,3.0,15906075118
30342,90-180天,117412985,11.0,13472290404
30343,30-90,86498277,20.0,18742522905
30344,30-90,30642960,6.0,18842211875


In [53]:
df_sample = pd.DataFrame(df_sample[['distinct_id','phone_number']])

In [54]:
df_sample

,distinct_id,phone_number
228861,149598890,13913899872
556389,103303226,13601020320
158245,123012545,15852469399
935799,130874531,15765002702
1008329,130406973,15283000648
...,...,...
148757,59474476,15906075118
192283,117412985,13472290404
47971,86498277,18742522905
654783,30642960,18842211875


In [55]:
df_sample.reset_index(drop=True).to_clipboard()